In [18]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2011 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2011 PUMS data

In [19]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TEN","TAXP","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2011_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan


In [20]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan


## Part A2. Assign county id to each puma

In [21]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part A3. Bring in and clean, reformat county AMIs data

In [24]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2011_amis.csv", delimiter=",", usecols=columns, encoding='latin1')
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,19400,22200,27700,32150,36600
1,Alameda County\n4-Per,Very Low Income,32350,36950,46150,53550,60950
2,Alameda County\n4-Per,Lower Income,45500,52000,64950,75350,85750
3,Alameda County\n4-Per,Median Income,64600,73850,92300,107050,121850
4,Alameda County\n4-Per,Moderate Income,77550,88600,110750,128450,146200
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12350,14100,17600,20450,23250
286,Yuba County\n4-Person,Very Low Income,20550,23450,29300,34000,38700
287,Yuba County\n4-Person,Lower Income,32850,37550,46900,54450,61950
288,Yuba County\n4-Person,Median Income,41000,46900,58600,68000,77350


In [25]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,19400,22200,27700,32150,36600
1,Alameda,Very Low Income,32350,36950,46150,53550,60950
2,Alameda,Lower Income,45500,52000,64950,75350,85750
3,Alameda,Median Income,64600,73850,92300,107050,121850
4,Alameda,Moderate Income,77550,88600,110750,128450,146200
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12350,14100,17600,20450,23250
286,Yuba,Very Low Income,20550,23450,29300,34000,38700
287,Yuba,Lower Income,32850,37550,46900,54450,61950
288,Yuba,Median Income,41000,46900,58600,68000,77350


In [26]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,485.00,555.00,692.50,803.75,915.00
5,Alpine,371.25,425.00,530.00,615.00,700.00
10,Amador,351.25,401.25,501.25,582.50,662.50
15,Butte,303.75,347.50,433.75,503.75,573.75
20,Calaveras,356.25,406.25,507.50,588.75,670.00
25,Colusa,301.25,343.75,428.75,497.50,566.25
30,Contra Costa,485.00,555.00,692.50,803.75,915.00
35,Del Norte,301.25,343.75,428.75,497.50,566.25
40,El Dorado,395.00,451.25,563.75,655.00,745.00
45,Fresno,301.25,343.75,428.75,497.50,566.25


In [27]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,808.75,923.75,1153.75,1338.75,1523.75
6,Alpine,618.75,706.25,882.50,1023.75,1165.00
11,Amador,585.00,668.75,835.00,968.75,1102.50
16,Butte,507.50,580.00,723.75,840.00,956.25
21,Calaveras,592.50,676.25,845.00,981.25,1116.25
26,Colusa,501.25,572.50,715.00,830.00,945.00
31,Contra Costa,808.75,923.75,1153.75,1338.75,1523.75
36,Del Norte,501.25,572.50,715.00,830.00,945.00
41,El Dorado,657.50,751.25,938.75,1090.00,1240.00
46,Fresno,501.25,572.50,715.00,830.00,945.00


In [28]:
li_df = df[df.Income_Category =="Lower Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1137.50,1300.00,1623.75,1883.75,2143.75
7,Alpine,988.75,1130.00,1412.50,1638.75,1865.00
12,Amador,936.25,1070.00,1336.25,1551.25,1765.00
17,Butte,811.25,926.25,1157.50,1343.75,1528.75
22,Calaveras,946.25,1081.25,1351.25,1567.50,1783.75
27,Colusa,801.25,915.00,1143.75,1327.50,1510.00
32,Contra Costa,1137.50,1300.00,1623.75,1883.75,2143.75
37,Del Norte,801.25,915.00,1143.75,1327.50,1510.00
42,El Dorado,1052.50,1202.50,1502.50,1743.75,1983.75
47,Fresno,801.25,915.00,1143.75,1327.50,1510.00


In [29]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1615.00,1846.25,2307.50,2676.25,3046.25
8,Alpine,1462.50,1672.50,2090.00,2425.00,2758.75
13,Amador,1168.75,1336.25,1670.00,1937.50,2205.00
18,Butte,1013.75,1157.50,1447.50,1678.75,1911.25
23,Calaveras,1203.75,1376.25,1720.00,1995.00,2270.00
28,Colusa,1001.25,1143.75,1430.00,1658.75,1887.50
33,Contra Costa,1615.00,1846.25,2307.50,2676.25,3046.25
38,Del Norte,1001.25,1143.75,1430.00,1658.75,1887.50
43,El Dorado,1313.75,1502.50,1877.50,2177.50,2478.75
48,Fresno,1001.25,1143.75,1430.00,1658.75,1887.50


In [30]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1938.75,2215.00,2768.75,3211.25,3655.00
9,Alpine,1755.00,2006.25,2507.50,2908.75,3310.00
14,Amador,1402.50,1602.50,2003.75,2323.75,2645.00
19,Butte,1216.25,1390.00,1737.50,2015.00,2293.75
24,Calaveras,1445.00,1651.25,2063.75,2393.75,2723.75
29,Colusa,1201.25,1372.50,1716.25,1991.25,2265.00
34,Contra Costa,1938.75,2215.00,2768.75,3211.25,3655.00
39,Del Norte,1201.25,1372.50,1716.25,1991.25,2265.00
44,El Dorado,1576.25,1802.50,2252.50,2612.50,2973.75
49,Fresno,1201.25,1372.50,1716.25,1991.25,2265.00


In [31]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,485.00,555.00,692.50,803.75,915.00,808.75,923.75,1153.75,1338.75,1523.75,1137.50,1300.00,1623.75,1883.75,2143.75,1615.00,1846.25,2307.50,2676.25,3046.25,1938.75,2215.00,2768.75,3211.25,3655.00
1,Alpine,371.25,425.00,530.00,615.00,700.00,618.75,706.25,882.50,1023.75,1165.00,988.75,1130.00,1412.50,1638.75,1865.00,1462.50,1672.50,2090.00,2425.00,2758.75,1755.00,2006.25,2507.50,2908.75,3310.00
2,Amador,351.25,401.25,501.25,582.50,662.50,585.00,668.75,835.00,968.75,1102.50,936.25,1070.00,1336.25,1551.25,1765.00,1168.75,1336.25,1670.00,1937.50,2205.00,1402.50,1602.50,2003.75,2323.75,2645.00
3,Butte,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
4,Calaveras,356.25,406.25,507.50,588.75,670.00,592.50,676.25,845.00,981.25,1116.25,946.25,1081.25,1351.25,1567.50,1783.75,1203.75,1376.25,1720.00,1995.00,2270.00,1445.00,1651.25,2063.75,2393.75,2723.75
5,Colusa,301.25,343.75,428.75,497.50,566.25,501.25,572.50,715.00,830.00,945.00,801.25,915.00,1143.75,1327.50,1510.00,1001.25,1143.75,1430.00,1658.75,1887.50,1201.25,1372.50,1716.25,1991.25,2265.00
6,Contra Costa,485.00,555.00,692.50,803.75,915.00,808.75,923.75,1153.75,1338.75,1523.75,1137.50,1300.00,1623.75,1883.75,2143.75,1615.00,1846.25,2307.50,2676.25,3046.25,1938.75,2215.00,2768.75,3211.25,3655.00
7,Del Norte,301.25,343.75,428.75,497.50,566.25,501.25,572.50,715.00,830.00,945.00,801.25,915.00,1143.75,1327.50,1510.00,1001.25,1143.75,1430.00,1658.75,1887.50,1201.25,1372.50,1716.25,1991.25,2265.00
8,El Dorado,395.00,451.25,563.75,655.00,745.00,657.50,751.25,938.75,1090.00,1240.00,1052.50,1202.50,1502.50,1743.75,1983.75,1313.75,1502.50,1877.50,2177.50,2478.75,1576.25,1802.50,2252.50,2612.50,2973.75
9,Fresno,301.25,343.75,428.75,497.50,566.25,501.25,572.50,715.00,830.00,945.00,801.25,915.00,1143.75,1327.50,1510.00,1001.25,1143.75,1430.00,1658.75,1887.50,1201.25,1372.50,1716.25,1991.25,2265.00


## Part A4. Add count dummies for each unit based on income and unit type

In [32]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [33]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [34]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
154450,H,2405,112.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,400000.00,nan,nan,94150.00,5.00,39.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
154451,H,4500,152.00,2.00,7.00,nan,nan,900.00,nan,nan,1.00,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# Part B2. Add count dummies for each unit based on income and unit type

In [35]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154448,H,4900,81.00,3.00,2.00,nan,nan,nan,nan,2.00,nan,700000.00,nan,nan,70700.00,7.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154449,H,1503,162.00,3.00,2.00,nan,nan,nan,nan,4.00,nan,nan,nan,nan,75000.00,7.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

# Part C1. Multiply each dummy by the the household weight variable

In [36]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [37]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
2018,H,6102,310.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3500.00,24.00,175000.00,1.00,nan,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,395.00,451.25,563.75,655.00,745.00,657.50,751.25,938.75,1090.00,1240.00,1052.50,1202.50,1502.50,1743.75,1983.75,1313.75,1502.50,1877.50,2177.50,2478.75,1576.25,1802.50,2252.50,2612.50,2973.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,310.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18884,H,6702,65.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3540.00,19.00,223700.00,5.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,395.00,451.25,563.75,655.00,745.00,657.50,751.25,938.75,1090.00,1240.00,1052.50,1202.50,1502.50,1743.75,1983.75,1313.75,1502.50,1877.50,2177.50,2478.75,1576.25,1802.50,2252.50,2612.50,2973.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,65.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
45746,H,6702,262.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3577.00,25.00,171000.00,4.00,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,395.00,451.25,563.75,655.00,745.00,657.50,751.25,938.75,1090.00,1240.00,1052.50,1202.50,1502.50,1743.75,1983.75,1313.75,1502.50,1877.50,2177.50,2478.75,1576.25,1802.50,2252.50,2612.50,2973.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,262.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
63250,H,5500,268.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3537.00,18.00,236000.00,1.00,nan,Napa CA,NaN,NaN,NaN,NaN,NaN,NaN,Napa,446.25,510.00,637.50,740.00,842.50,743.75,850.00,1062.50,1232.50,1402.50,1151.25,1315.00,1643.75,1907.50,2170.00,1487.50,1700.00,2125.00,2465.00,2805.00,1785.00,2040.00,2550.00,2957.50,3366.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,268.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
70434,H,6102,359.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3787.00,30.00,150000.00,1.00,nan,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,395.00,451.25,563.75,655.00,745.00,657.50,751.25,938.7

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [38]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,800,55.00,3.00,2.00,nan,1800.00,nan,nan,1.00,nan,100000.00,nan,nan,74000.00,4.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,6113,36.00,1.00,5.00,nan,nan,750.00,nan,3.00,nan,nan,780.00,32.00,29300.00,6.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,800,54.00,3.00,2.00,nan,nan,1000.00,nan,3.00,nan,nan,1150.00,101.00,12600.00,4.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,8004,9.00,3.00,2.00,nan,3000.00,nan,nan,1.00,nan,550000.00,nan,nan,96200.00,2.00,65.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,900,72.00,4.00,2.00,nan,nan,nan,nan,2.00,nan,260000.00,nan,nan,48004.00,5.00,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [39]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
13,H,3901,168.00,3.00,2.00,nan,510.00,nan,nan,1.00,nan,135000.00,nan,nan,37000.00,1.00,22.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
18,H,5420,114.00,2.00,2.00,nan,nan,1900.00,nan,3.00,nan,nan,2300.00,9.00,307000.00,1.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
19,H,1403,48.00,3.00,2.00,nan,1900.00,nan,nan,1.00,nan,300000.00,nan,nan,100050.00,1.00,52.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
31,H,2709,78.00,4.00,3.00,250.00,1500.00,nan,nan,1.00,nan,550000.00,nan,nan,60000.00,1.00,62.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
43,H,7802,70.00,2.00,7.00,nan,nan,850.00,nan,3.00,nan,nan,1000.00,25.00,47600.00,1.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Part D2. Aggregating at the PUMA level

In [40]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
200,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
300,55.00,1166.00,2166.00,642.00,334.00,662.00,3581.00,8285.00,4839.00,1257.00,1174.00,4096.00,9255.00,6339.00,1896.00,1414.00,4262.00,9559.00,6699.00,1896.00,1414.00,4262.00,9559.00,6816.00,1896.00,376.00,66.00,97.00,0.00,51.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
400,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
500,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8113,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8114,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8115,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [41]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [42]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
200,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
300,0.00,228.00,174.00,0.00,101.00,109.00,1226.00,2212.00,1919.00,393.00,551.00,1415.00,2762.00,3075.00,838.00,791.00,1415.00,2843.00,3226.00,838.00,791.00,1415.00,2843.00,3226.00,838.00,376.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
400,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
500,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8113,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8114,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8115,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [43]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")